In [1]:
import numpy as np
import ctypes as ct
import sys

In [90]:
DIM = 3  # Dimensionality

class TreeNode_ctype(ct.Structure):
    pass

class Cell_ctype(ct.Structure):
    _fields_ = [
        ("prob", ct.c_float),
        ("v", ct.c_float * DIM),
        ("u", ct.c_float * DIM),
        ("w", ct.c_float * DIM),
        ("ctu", ct.c_float * DIM),
        ("state", ct.c_int * DIM),
        ("i_nodes", ct.POINTER(TreeNode_ctype) * DIM),
        ("k_nodes", ct.POINTER(TreeNode_ctype) * DIM),
        ("dcu", ct.c_float),
        ("cfl_dt", ct.c_float),
        ("new_f", ct.c_int),
        ("ik_f", ct.c_int),
        ("del_f", ct.c_int)
    ]

TreeNode_ctype.fields = [
    ("key", ct.c_int),
    ("cell", Cell_ctype),
    ("left", ct.POINTER(TreeNode_ctype)),
    ("right", ct.POINTER(TreeNode_ctype))
]

class Cell:
    def __init__(self, PROB, V, U, W, CTU, STATE, DCU, CFL_DT, NEW_F, IK_F, DEL_F):
        I_NODES   = (ct.POINTER(TreeNode_ctype) * DIM)()
        K_NODES   = (ct.POINTER(TreeNode_ctype) * DIM)()
        V         = (ct.c_float * DIM)(*V)
        U         = (ct.c_float * DIM)(*U)
        W         = (ct.c_float * DIM)(*W)
        CTU       = (ct.c_float * DIM)(*CTU)
        STATE     = (ct.c_int * DIM)(*STATE)
        self.cell = Cell_ctype(prob = PROB, v = V, u = U, w = W, ctu = CTU, state = STATE, i_nodes = I_NODES, k_nodes = K_NODES, dcu = DCU, cfl_dt = CFL_DT, new_f = NEW_F, ik_f = IK_F, del_f = DEL_F)

class TreeNode:
    def __init__(self, KEY, CELL):
        LEFT  = (ct.POINTER(TreeNode_ctype))()
        RIGHT = (ct.POINTER(TreeNode_ctype))()
        self.treenode = TreeNode_ctype(key = KEY, cell = CELL, left = LEFT, right = RIGHT)

class Grid:
    def __init__(self):
        self.thresh = None
        self.dt     = None
        self.epoch  = None
        self.dx     = None
        self.xh     = None

class Traj: 
    sigma = None
    b     = None
    r     = None
    T     = None

class Measurement:
    def __init__(self):
        self.mean = None
        self.cov  = None

In [53]:
class BST:
    def __init__(self):
        self.root = None
        self.dead = None
        self.a_count = 0
        self.tot_count = 1
        self.cfl_min_dt = sys.float_info.max


    def insert(self, node):
        if self.root is None:
            self.root = node
        else:
            self._insert_recursive(self.root, node)

    def _insert_recursive(self, root, node):
        if node.key < root.key:
            if root.left is None:
                root.left = node
            else:
                self._insert_recursive(root.left, node)
        elif node.key > root.key:
            if root.right is None:
                root.right = node
            else:
                self._insert_recursive(root.right, node)

    def search(self, key):
        return self._search_recursive(self.root, key)

    def _search_recursive(self, root, key):
        if root is None or root.key == key:
            return root
        if key < root.key:
            return self._search_recursive(root.left, key)
        else:
            return self._search_recursive(root.right, key)
    
    def print(self):
        return self._print_recursive(self.root)

    def _print_recursive(self, root):
        if root is None:
            return 
        else:
            print(root.key)
            self._print_recursive(root.left)
            self._print_recursive(root.right)

In [31]:
def rosenberg_pair(state, d, m):
    if d==1:
        return state[0]
    
    new_state = np.empty(d-1)
    for i in range(d-1):
        new_state[i] = state[i]

    new_m = np.max(new_state)
    return rosenberg_pair(new_state, d-1, new_m) + m**d + (m - state[d-1])*((m + 1)**(d - 1) - m**(d - 1))

def state_conversion(state, DIM):
    shift_state = np.empty(DIM)
    for i in range(DIM):
        if(state[i] < 0):
            shift_state[i] = -2*state[i] - 1
        else:
            shift_state[i] = 2*state[i]

    m = np.max(shift_state)
    return rosenberg_pair(shift_state, DIM, m)

def mc(th):
    return max(0, min((1 + th)/2, 2.0, 2*th))

def get_size(root):
    if root == None:
        return 0
    
    return 1 + get_size(root.left) + get_size(root.right)

In [97]:
test_cell = Cell(0, [0.0,0.0,0.0], [0.0,0.0,0.0], [0.0,0.0,0.0], [0.0,0.0,0.0], [0,0,0], 0, 0, 0, 0, 0)
test_node = TreeNode(0, test_cell)
print(test_node.treenode.cell.cell.prob)
test_cell.cell.prob = 1
print(test_node.treenode.cell.cell.prob)

0.0
1.0


In [4]:
#========================================== Begin User Input ===============================================#
print( "Reading in user inputs...\n" )

NM        = 1          # Number of measurements 
FILE_PATH = "./Epochs" # Measurement file path

G           = Grid()   # Grid instance
G.thresh    = 2E-5     # Probability threshold
OUTPUT      = True     # Write info to terminal
RECORD      = True     # Write PDFs to .txt file
MEASURE     = True     # Take discrete measurement updates
OUTPUT_FREQ = 20       # Number of steps per output to terminal
DEL_STEP    = 25       # Number of steps per deletion procedure
NUM_DIST    = 6        # Number of distributions recorded per measurement
#===========================================================================================================#

Reading in user inputs...



In [5]:
#===================================== Read in measurement/trajectory info =================================#
print("Reading in discrete measurements...\n")

measurement_file = open(FILE_PATH + "/measurements.txt", "r")
mean = np.empty(DIM)
cov  = np.empty((DIM,DIM))

measurement_file.readline(); # Skip label line
mean = [float(string) for string in measurement_file.readline().split(" ")] # Mean
measurement_file.readline(); # Skip blank space
measurement_file.readline(); # Skip label line
for i in range(DIM):
    cov[i] = [float(string) for string in measurement_file.readline().split(" ")] # Covariance
G.epoch = mean
G.dx = [x/2 for x in np.diag(cov)] # Grid width
G.xh = [x/2 for x in G.dx]         # Half grid width

Lor = Traj() 
measurement_file.readline(); # Skip blank space
measurement_file.readline(); # Skip label line
Lor.sigma = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.b = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.r = float(measurement_file.readline())
measurement_file.readline(); # Skip label line
Lor.T = float(measurement_file.readline())

measure_time = Lor.T/NM                 # Time between measurements
record_time = measure_time/(NUM_DIST-1) # Time between recording PDF
measurement_file.close()
#===========================================================================================================#

Reading in discrete measurements...



In [6]:
print(Lor.sigma)

4.0
